# Conexión a Azure Quantum y envío del trabajo

Suponemos que ya hemos realizado todos los pasos para usar AzureQPU:

    - Crear una cuenta en Azure
    - Suscribir un programa gratuito o de pago
    - Crear un servicio de Azure Quantum creando previamente un espacio de trabajo. 

Hecho esto ya solo tendría que establecer la conexión mediante mi identificador de recurso y localización de ejecución. 

    - MyResourceID
    - MyLocation


**Nota:** En el documento CREACIÓN DE CUENTA AZURE QUANTUM se explica cómo crear un área de trabajo:

https://mega.nz/file/MiMlCKJA#l1VMMoEwtq3UHLlgiCJiu4nSMEgS31V_cw0zY2tJobQ

## Instalación del Azure Quantum Developer Kit 


Podemos instalar el QDK de Azure Quantum en Python para Q#, Qiskit, Cirq:

https://learn.microsoft.com/es-es/azure/quantum/install-overview-qdk

Para Cirq:

%pip install qsharp azure-quantum[cirq]    

## Envío de un programa Cirq a Azure Quantum

https://learn.microsoft.com/es-es/azure/quantum/quickstart-microsoft-cirq?tabs=tabid-ionq&pivots=platform-local

In [ ]:
%pip install --upgrade azure-quantum[cirq]

In [2]:
import cirq
from azure.quantum import Workspace
from azure.quantum.cirq import AzureQuantumService

In [3]:
workspace = Workspace(
    resource_id = "/subscriptions/0f4027d3-276d-4941-ad53-a9244c367728/resourceGroups/imardiegoAzureQuantumResourcesGroup/providers/Microsoft.Quantum/Workspaces/imardiegoAzureQuantumWorkArea",
    location = "westeurope"
)
service = AzureQuantumService(workspace)

In [4]:
q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CNOT(q0, q1),
    cirq.measure(q0, q1, key='resultados')
)
print(circuit)

0: ───H───@───M('resultados')───
          │   │
1: ───────X───M─────────────────


In [5]:
print("Targets disponibles en tu área de trabajo:")
for target in service.targets():
    print(target)
    
# debo hacer clic en la url y validarme en azure quantum

Targets disponibles en tu área de trabajo:


gio: https://login.microsoftonline.com/235bdb0d-7ee9-4049-ae95-0f2fb6253595/oauth2/v2.0/authorize?client_id=04b07795-8ddb-461a-bbee-02f9e1bf7b46&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8400&scope=https%3A%2F%2Fquantum.microsoft.com%2F.default+offline_access+openid+profile&state=UbaSWgwpBjNGhvXq&code_challenge=65LowJQGw6j2S9BhSRED3KI-9a6_qdJsiLvEIzdQO58&code_challenge_method=S256&nonce=1718760bf029d920d195d11740d12dd7db024f5b959c7391eaff96b57e57c1da&client_info=1&prompt=select_account: Operation not supported


<Target name="quantinuum.sim.h1-1sc", avg. queue time=8 s, TargetAvailability.AVAILABLE>
<Target name="quantinuum.sim.h1-1e", avg. queue time=18 s, TargetAvailability.AVAILABLE>


In [6]:
# Enviar y ejecutar
result = service.run(
    program=circuit,
    repetitions=10,
    target="quantinuum.sim.h1-1sc",
    name="MiPrimerJobCirq"
)

# Debo hacer clic en la url y validar en azure quantum

gio: https://login.microsoftonline.com/235bdb0d-7ee9-4049-ae95-0f2fb6253595/oauth2/v2.0/authorize?client_id=04b07795-8ddb-461a-bbee-02f9e1bf7b46&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8400&scope=https%3A%2F%2Fquantum.microsoft.com%2F.default+offline_access+openid+profile&state=CPMsTHigGUSdxNAy&code_challenge=VFCt2TgZnGIrrMBz28v78EkuuIiraK80T-kedbDQMJw&code_challenge_method=S256&nonce=1f4c9137f1f85438d3a8193df4ef7b247412c62a072360df42f54aa6145013c0&client_info=1&prompt=select_account: Operation not supported


.......

In [7]:
# Accede a las mediciones por la clave definida en el circuito
measurements = result.measurements['resultados']

# Ejemplo: mostrar las 5 primeras mediciones
print(measurements[:15])

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]


In [8]:
import numpy as np
unique, counts = np.unique(measurements, axis=0, return_counts=True)
for u, c in zip(unique, counts):
    print(f"{u}: {c}")

[0]: 10


### Envío de programa Cirq a través del simulador

https://learn.microsoft.com/es-es/azure/quantum/quickstart-microsoft-cirq?tabs=tabid-ionq&pivots=platform-simulator

**Hacerlo!!!!**